## 参数统计
```python
DATA_PATH = "../cache/layer29.pt"
SAVE_DIR = "attention_analysis/layer29"
SAVE_SVG = True
```

In [1]:
import os
import torch
import papermill as pm
from tqdm.auto import tqdm

In [ ]:
data_paths = [f"../cache/lake_mist/layer{i}.pt" for i in range(30)]
data_example = torch.load('../cache/lake_mist/layer3.pt', map_location='cpu', weights_only=True)

In [ ]:
def print_info(data_eg):
    print('=' * 30 + " keys list " + '=' * 30)
    print('\n'.join(data_eg.keys()))
    print('=' * 28 + " keys content " + '=' * 28)
    for key in data.keys():
        if isinstance(data_eg[key], torch.Tensor):
            print(f"Tensor '{key}', dim: {data_eg[key].shape}")
        else:
            print(f"key: '{key}' content type: {type(data_eg[key])}")

In [ ]:
print_info(data_example)

In [ ]:


for i, path in enumerate(tqdm(data_paths, desc="Total Progress")):
    output_notebook = f"runs/result_layer_{i}.ipynb"
    os.makedirs("runs", exist_ok=True)
    
    try:
        pm.execute_notebook(
            'extract_all_attention.ipynb',
            output_notebook,
            parameters={
                'DATA_PATH': path,
                'SAVE_DIR': f"attention_analysis/lake_mist/layer{i}",
                'SAVE_SVG': True
            },
            progress_bar=True
        )
    except KeyboardInterrupt:
        print("\n[!] 用户手动停止了循环。")
        break
    except Exception as e:
        print(f"\n[X] 处理 {path} 时出错: {e}")
        continue

Total Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]